In [1]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from sbi_scraping import get_own_stock_df
from minkabu_scraping import append_dividend
from tse_stock_list_downloader import append_market_and_industries

import config

import pandas as pd
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
# 通常のドライバ
driver = webdriver.Chrome(
    service=ChromeService(ChromeDriverManager().install())
)

In [ ]:
try:
    # 証券会社のwebサイトから保有株情報を抽出する
    df_own_stock = get_own_stock_df(driver)
except Exception as e:
    print("Failed get_own_stock_df")
    print(str(e))
finally:
    driver.quit()

In [ ]:
try:
    # 配当金額を抽出する（処理に時間がかかる）
    df_own_stock = append_dividend(df_own_stock)
    # このあとでエラーになった場合途中から再開できるよう、CSVに吐き出しておく
    df_own_stock.to_csv("own_stock.csv")
except Exception as e:
    print("Failed get_own_stock_df")
    print(str(e))

In [23]:
# 吐き出しておいたCSVを読み込む
df_own_stock = pd.read_csv("own_stock.csv", index_col=0)

In [25]:
# データフレームに市場と業種を入れる
df_own_stock = append_market_and_industries(df_own_stock)

# スプレッドシートに書き込めるようにリスト形式に変換する
list_own_stock = df_own_stock.values.tolist()

In [16]:
# 2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# 認証情報設定
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "keys/credentials.json", scope
)

In [17]:
# OAuth2の資格情報を使用してGoogle APIにログイン
gc = gspread.authorize(credentials)

In [18]:
# 共有設定したスプレッドシートを開く
spreadsheet = gc.open_by_key(config.SPREADSHEET_KEY)

In [19]:
# シート名を指定してシートを開く
worksheet = spreadsheet.worksheet("②高配当株ポートフォリオ")

In [ ]:
# シートの既存の値をクリアする（B列からJ列だけ上書きするため）
worksheet.batch_clear(["B4:I300"])

In [ ]:
# シートのB4:J4からデータフレームの値を書き込む
cell_range = f"B4:I{3 + len(list_own_stock)}"
worksheet.update(cell_range, list_own_stock)